# **Simple RAG Application Using LangChain And Qwen2.5**

In [1]:
# Install the nessary packeges
!pip install langchain -qU
!pip install langchain -qU
!pip install langchain_community langchain
!pip install langchain_chroma
!pip install git+https://github.com/huggingface/transformers@v4.56.0-Embedding-Gemma-preview

  Using cached langchain_chroma-1.0.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached chromadb-1.3.5-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.2 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (8.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.3 MB/s eta 0:00:00
   

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import subprocess
subprocess.Popen(['ollama','serve'])
print("<<Ollama server launched sucessfully>>")

<<Ollama server launched sucessfully>>


In [4]:
!pip install ollama

**Initialize the LLM**

In [5]:
from langchain_community.llms import Ollama
import ollama
# prompt ="what is the  prompt engineering ?"

# response =ollama.chat(model='deepseek-r1:1.5b',messages=[{"role":"user","content":prompt}])
# print(response['message']['content'])
ollama.pull(model='qwen2.5:7b') # run at once
llm =Ollama(model='qwen2.5:7b')

/tmp/ipython-input-1168748708.py:8: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm =Ollama(model='qwen2.5:7b')


**Initialize the Embedding Model**

In [6]:
# !pip install -q langchain-qwen3
!pip install -U langchain-nomic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 7.7 MB/s eta 0:00:00


In [ ]:
# from langchain_qwen3 import Qwen3Embeddings
# embeddings_modle = Qwen3Embeddings(
#     model="Qwen/Qwen3-Embedding-0.6B" # HF model name
#     )


In [7]:
!nomic login


                        Authenticate with the Nomic API                         
                        https://atlas.nomic.ai/cli-login                        
  Click the above link to retrieve your access token and then run `nomic login  
                                    [token]`                                    


In [20]:
from google.colab import userdata
KEY=userdata.get('KEY')

import os
os.environ["NOMIC_API_KEY"] = KEY

from langchain_nomic import NomicEmbeddings

embeddings_model = NomicEmbeddings(model="nomic-embed-text-v1.5")


**Create and Embed Documents**

In [9]:

from langchain_core.documents import Document

# Define a list of documents with content and metadata
documents = [
    Document(
        page_content="The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.",
        metadata={"source": "cricket news"},
    ),
    Document(
        page_content="The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.Both ties promise thrilling encounters, featuring some of the best talents in world football.",
        metadata={"source": "football news"},
    ),
    Document(
        page_content="As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives.",
        metadata={"source": "election news"},
    ),
    Document(
        page_content="The AI revolution continues to transform industries and reshape the global economy.Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety.",
        metadata={"source": "ai revolution news"},
    ),
]


In [ ]:
# pip install langchain_chroma

In [10]:
# Create a vector store using the document and embedding model

from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings_model
)

**Perform Similarity Search**

In [11]:
results =vectorstore.similarity_search("test match")

for result in results :
  print("----------------------")
  print(result.page_content)
  print(result.metadata)

----------------------
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
{'source': 'cricket news'}
----------------------
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.Both ties promise thrilling encounters, featuring some of the best talents in world football.
{'source': 'f

In [12]:
results =vectorstore.similarity_search("Machine learning")

for result in results :
  print("----------------------")
  print(result.page_content)
  print(result.metadata)

----------------------
The AI revolution continues to transform industries and reshape the global economy.Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety.
{'source': 'ai revolution news'}
----------------------
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.Both ties promise thrilling encounters, featuring some of the best talents in world football.
{'source': 'football news'}
----------------------
As election season heats up, the latest developments reveal a highly competitive atmosphere a

**Create Retriever**

In [13]:
# create a retriever from the vector
retriever =vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs ={'k':1} # needed ouput count
)
#perform batch retrieval using the retriver

batch_results =retriever.batch(["machine learning"])

for result in batch_results:
  print("------------------")
  for doc in result:
    print(doc.page_content)
    print(doc.metadata)

------------------
The AI revolution continues to transform industries and reshape the global economy.Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety.
{'source': 'ai revolution news'}


In [14]:
# create a retriever from the vector
retriever =vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs ={'k':1} # needed ouput count
)
#perform batch retrieval using the retriver

batch_results =retriever.batch(["machine learning","test match"])

for result in batch_results:
  print("------------------")
  for doc in result:
    print(doc.page_content)
    print(doc.metadata)

------------------
The AI revolution continues to transform industries and reshape the global economy.Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety.
{'source': 'ai revolution news'}
------------------
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
{'source': 'cricket news'}


**Create Prompt Template**

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define a message template for the chatbot

message = """
Answer this question the provideed context only.

{question}

contex:
{context}
"""

# Create a chat prompt template from the message
prompt =ChatPromptTemplate.from_messages(
    [
        "human" ,message
    ]
)

**Chain Retriever and Prompt Template with LLM**

In [16]:
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [ ]:
# response = chain.invoke({
#     "question": "current state of 2023 t20 world cup"
# })

# print(response.content)

In [17]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

chain = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


In [18]:
response = chain.invoke({"question": "current state of 2024 t20 world cup"})
print(response)


The T20 World Cup 2024 is currently underway, providing thrilling matches for cricket enthusiasts globally. India's team, led by Rohit Sharma, faces an important match against Ireland, with Jasprit Bumrah anticipated to be a key player in their efforts. Additionally, there has been controversy surrounding the pitch conditions at Nassau County International Cricket Stadium in New York, which received criticism after a low-scoring game between Sri Lanka and South Africa.
